# Decision Trees implemented with Entropy

In [1]:
import numpy as np
from collections import Counter

### Entropy is defined as:

E = - $\sum$ p(X) . $log_{2}$(p(X))

In [2]:
def entropy(y):
    """
    Measures the uncertainty of labels based on a feature 
    """
    # First, count the occurences of each label in the dataset
    hist = np.bincount(y)
    # Proportion of each label in the dataset
    ps = hist / len(y)
    # Formula for entropy (if condition is to avoid negative numbers inside log2() function)
    return -np.sum([p*np.log2(p) for p in ps if p > 0])

### Helper Node class to build the tree

In [3]:
class Node:
    
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, *, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        # If a node has a value (i.e. a corresponding class label), then it is the leaf node
        return self.value is not None    
        

### Decision Tree Class

In [4]:
class DecisionTree:
    """
    Here, the tree needs some stopping criteria as to when to stop building the tree:
        1. When there is only one label present in the remaining data
        2. When the number of samples is less than 2
        3. When a max_depth is reached and we don't want to go further
    So, these are the stopping criteria that we can check to stop building the tree further and make that node as a leaf node
    
    X.shape = (n_samples, n_features)
    """
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
        
    def fit(self, X, y):
        # Grow the tree while training based on the based split features
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)
    
    def predict(self, X):
        # Traverse the tree to get a class for the feature values provided
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _grow_tree(self, X, y, depth=0):
        # A helper function to grow the tree recursively
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # Check for a stopping criteria (base condition)
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            # This means we are at the leaf node
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # Randomly selected indices for features to be selected
        feature_indices = np.random.choice(n_features, self.n_features, replace=False)
        
        # Do a greedy search to build the tree
        
        # 1. First get the best feature to split the data
        best_feature, best_threshold = self._best_criteria(X, y, feature_indices)
        
        # 2. Split the data using the best feature
        left_indices, right_indices = self._split(X[:, best_feature], best_threshold)
        
        # 3. Recursively grow the tree
        left = self._grow_tree(X[left_indices, :], y[left_indices], depth+1)
        right = self._grow_tree(X[right_indices, :], y[right_indices], depth+1)
        
        # 4. Return the node with its values and left and right children
        return Node(best_feature, best_threshold, left, right)
        
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)
        return most_common[0][0]
    
    def _best_criteria(self, X, y, feature_indices):
        # Calculate the information gain by going through all the features
        best_gain = -1
        split_index, split_threshold = None, None
        for feature_index in feature_indices:
            X_col = X[:, feature_index]
            thresholds = np.unique(X_col)
            for thres in thresholds:
                gain = self._get_information_gain(y, X_col, thres)
                
                if gain > best_gain:
                    best_gain = gain
                    split_index = feature_index
                    split_threshold = thres
                    
        return split_index, split_threshold
    
    def _get_information_gain(self, y, X_col, split_threshold):
        # Check formula to calculate the information gain above
        # 1. Parent entropy
        parent_entropy = entropy(y)
        
        # 2. Generate split
        left_indices, right_indices = self._split(X_col, split_threshold)
        if len(left_indices) == 0 or len(right_indices):
            # The node cannot be splitted anymore. So return an information gain = 0
            return 0
        
        # 3. Weighted average for each child
        N = len(y)
        n_left, n_right = len(left_indices), len(right_indices)
        left_entropy, right_entropy = entropy(y[left_indices]), entropy(y[right_indices])
        # Weighted child entropy
        child_entropy = n_left/N*left_entropy + n_right/N*right_entropy
        
        # 4. Information gain = parent_entropy - weighted_child_entropy
        information_gain = parent_entropy - child_entropy
        return information_gain
        
    def _split(self, X_col, split_threshold):
        left_indices = np.argwhere(X_col <= split_threshold).flatten()
        right_indices = np.argwhere(X_col > split_threshold).flatten()
        return left_indices, right_indices
    
    def _traverse_tree(self, x, node):
        # Starting from the root, traverse down the tree based on the features of each x
        # This is done recursively with the following base condition
        if node.is_leaf_node():
            return node.value
        
        # If the value of that feature (based on which the node splits) is less that or equal to the threshold of that node,
        # then traverse to the left of that node, else to the right
        # Here, node.feature_index gives the index of the feature in the data
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        
        return self._traverse_tree(x, node.right)
        

### Train and Test the Decision Tree model

In [5]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.6052631578947368
